This chapter was a doozie... lots of problems compiling and problems with error message suppresion in the jupyter framework. After fighting for an inordinate amount of time, the towel was thrown in and I'm moving on to the next damn chapter. Here's the trash from my struggles :) 

In [1]:
import scala.{Option => _, Either => _, _}

sealed trait Either[+E,+A] {
    def map[B](f: A => B): Either[E, B] = {
        this match {
            case Right(a) => Right(f(a))
            case Left(e) => Left(e)
        }
    }

    def flatMap[EE >: E, B](f: A => Either[EE, B]): Either[EE, B] = {
        this match {
            case Left(e) => Left(e)
            case Right(a) => f(a)
        }
    }

    def orElse[EE >: E, B >: A](b: => Either[EE, B]): Either[EE, B] = {
        this match {
            case Left(_) => b
            case Right(a) => Right(a)
        }
    }

    def map2[EE >: E, B, C](b: Either[EE, B])(f: (A, B) => C): Either[EE, C] = {
        for { a <- this; b1 <- b } yield f(a, b1)
    }
}

case class Left[+E](get: E) extends Either[E,Nothing]
case class Right[+A](get: A) extends Either[Nothing,A]

object Either {
    def traverse[E,A,B](es: List[A])(f: A => Either[E, B]): Either[E, List[B]] = {
        es match {
            case Nil => Right(Nil)
            case h::t => (f(h) map2 traverse(t)(f))(_::_)
        }
    }

    def sequence[E,A](es: List[Either[E,A]]): Either[E,List[A]] = {
        traverse(es)(x => x)
    }

    def mean(xs: IndexedSeq[Double]): Either[String, Double] = {
        if (xs.isEmpty) {
            Left("mean of empty list!")
        } else {
            Right(xs.sum / xs.length)
        } 
    }
    
    def safeDiv(x: Int, y: Int): Either[Exception, Int] = {
        try Right(x / y)
        catch { case e: Exception => Left(e) }
    }

    def Try[A](a: => A): Either[Exception, A] = {
        try Right(a)
        catch { case e: Exception => Left(e) }
    }
    
    def traverse_1[E,A,B](es: List[A])(f: A => Either[E, B]): Either[E, List[B]] = { 
        es.foldRight[Either[E,List[B]]](Right(Nil))((a, b) => f(a).map2(b)(_ :: _))
    }
}

import scala.{Option => _, Either => _, _}
defined trait Either
defined class Left
defined class Right
defined object Either

In [ ]:
import scala.{Option => _, Some => _, Either => _, _} // hide std library `Option`, `Some` and `Either`, since we are writing our own in this chapter

sealed trait Option[+A] {
    def map[B](f: A => B): Option[B] = {
        this match {
            case None => None
            case Some(a) => Some(f(a))
        }
    }

    def getOrElse[B>:A](default: => B): B = {
        this match {
            case None => default
            case Some(a) => a
        }
    }

    def flatMap[B](f: A => Option[B]): Option[B] = {
        map(f) getOrElse None
    }

    def orElse[B>:A](ob: => Option[B]): Option[B] = {
        this map (Some(_)) getOrElse ob
    }

    def filter(f: A => Boolean): Option[A] = {
        this match {
            case Some(a) if f(a) => this
            case _ => None
        }
    }
}
case class Some[+A](get: A) extends Option[A]
case object None extends Option[Nothing]

object Option {
    def failingFn(i: Int): Int = {
        val y: Int = throw new Exception("fail!") // `val y: Int = ...` declares `y` as having type `Int`, and sets it equal to the right hand side of the `=`.
        try {
          val x = 42 + 5
          x + y
    }
        catch { case e: Exception => 43 } // A `catch` block is just a pattern matching block like the ones we've seen. `case e: Exception` is a pattern that matches any `Exception`, and it binds this value to the identifier `e`. The match returns the value 43.
    }

    def failingFn2(i: Int): Int = {
        try {
          val x = 42 + 5
          x + ((throw new Exception("fail!")): Int) // A thrown Exception can be given any type; here we're annotating it with the type `Int`
        }
        catch { case e: Exception => 43 }
    }
    
    def mean(xs: Seq[Double]): Option[Double] =
        if (xs.isEmpty) None
        else Some(xs.sum / xs.length)
    
    // Exercise 2 
    def variance(xs: Seq[Double]): Option[Double] = {
        mean(xs) flatMap (mu => mean(xs.map(x => math.pow(x - mu, 2))))
    }

    // Exercise 3
    def map2[A,B,C](a: Option[A], b: Option[B])(f: (A, B) => C): Option[C] = {
        a flatMap (aa => b map (bb => f(aa, bb)))
    }

    // Exercise 4
    def sequence[A](a: List[Option[A]]): Option[List[A]] = {
        a match {
            case Nil => Some(Nil)
            case h :: t => h flatMap (hh => sequence(t) map (hh :: _))
        }
    }

    // Exercise 5
    def traverse[A, B](a: List[A])(f: A => Option[B]): Option[List[B]] = {
        a match {
            case Nil => Some(Nil)
            case h::t => map2(f(h), traverse(t)(f))(_ :: _)
        }
    }
}

In [ ]:
trait Partial[+E,+A] {
  def map[B](f:A => B): Partial[E,B] = this match {
    case Success(a) => Success(f(a))
    case Errors(e) => Errors(e)
  }
 
  def flatMap[EE >: E, B](f: A => Partial[EE, B]): Partial[EE, B] = this match {
    case Success(a) => f(a)
    case Errors(e) => Errors(e)
  }
 
  def map2[EE >: E, B, C](b: Partial[EE,B])(f: (A,B) => C): Partial[EE,C] = (this, b) match {
    case (Errors(e1), Errors(e2)) => Errors(e1 ++ e2)
    case (Errors(e), _) => Errors(e)
    case (_, Errors(e)) => Errors(e)
    case (Success(aa), Success(bb)) => Success(f(aa,bb))
  }
}
case class Errors[+E](get: Seq[E]) extends Partial[E,Nothing]
case class Success[+A](get: A) extends Partial[Nothing,A]
 
case class Person(name: Name, age: Age)
sealed class Name(val value: String)
sealed class Age(val value: Int)
 
def mkName(name: String): Partial[String, Name] =
  if (name == "" || name == null) Errors(Seq("Name is empty."))
  else Success(new Name(name))
 
def mkAge(age: Int): Partial[String, Age] =
  if (age < 0) Errors(Seq("Age is out of range."))
  else Success(new Age(age))
 
def mkPerson(name: String, age: Int): Partial[String, Person] =
  mkName(name).map2(mkAge(age))(Person(_, _))